In [1]:
from oxygen_analysis import group_channels, get_filenames, import_data
import numpy as np
import pandas as pd
import os
from nptdms import TdmsFile
import multiprocessing as mp
import plotly.graph_objects as go
import ipywidgets as widgets

In [2]:
# Get the filenames to load data from 
filepath = 'C:\\Users\\derek\\Desktop\\OxyPDE_10032019\\'
filenames = get_filenames(filepath)

In [3]:
def get_photo_data(filename):
    pd_data = []
    pdfile = TdmsFile(filename)
    temp_data = pdfile.group_channels(pdfile.groups()[0])
    for dc in temp_data:
        if 'time' in dc.path.lower():
            pass
        elif 'photo' in dc.path.lower() or 'coil' in dc.path.lower():
            pd_data.append(dc.as_dataframe().values)
    return np.asarray(pd_data, dtype=np.float64).squeeze()

In [4]:
def generate_x_values(data_shape):
    x_values = np.empty((min(data_shape), max(data_shape)))
    x_series = np.divide(list(range(max(data_shape))), max(data_shape))
    for i in range(min(trimmed_data.shape)):
        x_values[i, :] = x_series
    return x_values

In [5]:
# This code generates the data for one test now it needs to be put into a funtion that allows it
# to be used on all of the tests so we can look at all of the data at once
def get_useful_data(file_loc):
    data = get_photo_data(file_loc)
    lower_window_limit = 250
    upper_window_limit = 4750
    max_avg = int(data.argmax(axis=1)[:2].mean())
    window_min = int(max_avg-lower_window_limit)
    window_max = int(max_avg+upper_window_limit)
    remove_indicies = list(range(0, window_min)) + list(range(window_max, max(data.shape)))
    trimmed_data = np.delete(data, remove_indicies, axis=1).transpose()
    trimmed_x_values = np.divide(range(window_min, window_max), 1e6)
    return np.column_stack((trimmed_data, trimmed_x_values.reshape((max(trimmed_data.shape), 1))))

In [ ]:
# See if we can get all of the data
test_data = []
for f in filenames:
    test_data.append(get_useful_data(os.path.join(filepath, f)))
test_data = np.asarray(test_data)


In [9]:
axis_font = dict(size=24, family='DejaVu Sans')
# cond_fig = go.FigureWidget()
from plotly.subplots import make_subplots

test_slider = widgets.IntSlider(
    min=0,
    max=test_data.shape[0]-1,
    value=0,
    step=1,
    readout=True,
    description='Test_Number'
)
test_slider.layout.width='800px'

# cond_fig = make_subplots(specs=[[{"secondary_y": True}]])
cond_fig = make_subplots(rows=2,
                         cols=1,
                         shared_xaxes=True, 
                         specs=[[{"secondary_y": True}], [{"secondary_y": True}]],
                        vertical_spacing=0.2
                        )
cond_fig = go.FigureWidget(cond_fig)


test_number = 0
number_of_sensors = 4

# plot the first trace in the set of data where the first 4 datasets are the sensor traces and
# the last one is the time trace
time_trace = test_data[0, :, -1]
sensor_name = ['Photodiode 1', 'Photodiode 2', 'Photodiode 3', "Conductivity"]

def update_plot_traces(test_number):
    with cond_fig.batch_update():
        cond_fig.update_layout(title=go.layout.Title(text=filenames[test_number], xref="paper",x=0))
        time_trace = test_data[test_number, :, -1]
        traces = []
        for idx, trace in enumerate(cond_fig.update_traces()['data']):
            trace['x'] = test_data[test_number, :, -1]
            trace['y'] = test_data[test_number, :, idx]
    return None

for i in range(number_of_sensors):
    x_data = time_trace
    y_data = test_data[0, :, i]
    trace_name = sensor_name[i]
    if i==3:
        cond_fig.add_trace(
            go.Scattergl(
                x=x_data,
                y=y_data,
                name=trace_name
            ),
            row=2,
            col=1
        )
    else:
        cond_fig.add_trace(
            go.Scattergl(
                x=x_data,
                y=y_data,
                name=trace_name
            ),
            row=1,
            col=1
        )

vb = widgets.VBox([widgets.interactive(update_plot_traces, test_number=test_slider), cond_fig])
vb